# Creating a custom Layer Normalization layer 

## Importing libraries

In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from ipynb.fs.full.Useful_funcs import data_pipeline, pre_model
from sklearn.datasets import fetch_california_housing

## Loading dataset

In [5]:
housing = fetch_california_housing()

In [7]:
x_train, x_train_scaled, x_valid, x_valid_scaled, x_test, x_test_scaled, y_train, y_valid, y_test = data_pipeline(housing)

## Building the custom layer

In [14]:
class Layer_normalization(keras.layers.Layer):
    def __init__(self, eps = 0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps # smoothing term
    def build(self, batch_input_shape):
        self.alpha = self.add_weight(name = 'alpha', shape = batch_input_shape[-1:], initializer = keras.initializers.ones) # Initializing alpha
        self.beta = self.add_weight(name = 'beta', shape = batch_input_shape[-1:], initializer = keras.initializers.zeros) # Initializing beta
        super().build(batch_input_shape)
    def call(self, x):
        mean, variance = tf.nn.moments(x, axes = -1, keepdims = True) # Getting the mean and variance of the input instances
        return self.alpha * (x - mean) / (tf.sqrt(variance + self.eps)) + self.beta
    '''We have included eps inside tf.sqrt() because when the derivative of sqrt(z) is infiinity when z = 0. So the gradients will explode even if the
    variance has a single zero component. Adding eps inside the sqrt avoids it.'''
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'eps' : eps}

- We have to make sure that the perfomance of the custom layer is on par with the inbuilt LayerNormalization() layer from keras.
- For that we will measure the performance of the training data on each of the instances created from the class.

In [15]:
x = x_train.astype(np.float32)

In [16]:
custom_norm_layer = Layer_normalization() # Custom instance
keras_norm_layer = keras.layers.LayerNormalization() # Keras instance

In [17]:
tf.reduce_mean(keras.losses.mean_absolute_error(keras_norm_layer(x), custom_norm_layer(x)))

<tf.Tensor: shape=(), dtype=float32, numpy=5.6045884e-08>

- The error is really small. To be extra sure, we can set the values of alpha and beta as random and check again.

In [18]:
random_alpha = np.random.rand(x.shape[-1])
random_beta = np.random.rand(x.shape[-1])

In [19]:
# Setting the random alpha beta values for both the instances
custom_norm_layer.set_weights([random_alpha, random_beta])
keras_norm_layer.set_weights([random_alpha, random_beta])

In [20]:
tf.reduce_mean(keras.losses.mean_absolute_error(keras_norm_layer(x), custom_norm_layer(x)))

<tf.Tensor: shape=(), dtype=float32, numpy=2.163107e-08>

- We can find that the error is still very small. So we conclude the custom layer works fine.